# 10/31/2016: Session 6
# Attribute Access and Functional Programming

In [28]:
# recap of classes and iteration function __iter__

class Counter:
    def __init__(self, low, high):
        self.current = low
        self.high = high
    
    def __iter__(self): #
        return self
    
    def next(self):
        if self.current > self.high:
            raise StopIteration  #this is a built-in exception
        else:
            self.current += 1
            return self.current
    
    
for c in Counter(3,8):
    print c
    
print """
"""

obj = Counter(3,8)
print obj.current, "== current"
print obj.next()
print obj.next()
print obj.next()
print obj.next()
print obj.next()
print obj.next()

4
5
6
7
8
9


3 == current
4
5
6
7
8
9


In [ ]:
%run hello.py #this runs the script within python

#### Three pillars of object-oriented programming (OOP)
. Encapsulation
    - allows an object to control its own data
    - the integrity of an object's state (i.e., its attribute values) is maintained

. Inheritance
    - adding another tier of attribute lookup
    - e.g., from own classes, or from built-in classes

. Polymorphism
    - same-named method used on different object types achieving different results
    - e.g., adding (+ or __add__) strings vs numbers

In [48]:
class MyClass(object):

    # when read attribute does not exist
    def __getattr__(self, name):
        default = 0 # default sets 
        print 'getattr: "{}" not found; setting default'.format(name)
        setattr(self, name, default)
        return default

    # when attribute is read
    def __getattribute__(self, name):
        print 'getattribute:  attempting to access "{}"'.format(name)
        return object.__getattribute__(self, name) # calling parent attribute (object)
        '''
        object has attributes of __getattribute__
        '''

    # when attribute is assigned
    def __setattr__(self, name, value):
        print 'setattr:  setting "{}" to value "{}"'.format(name, value)
        self.__dict__[name] = value

x = MyClass()

x.a = 5             # setattr:  setting "a" to value "5"

print x.a           # getattribute:  attempting to access "__dict__"
                    # getattribute:  attempting to access "a"
                    # 5

print x.ccc         # getattribute:  attempting to access "ccc"
                    # getattr: "ccc" not found; setting default
                    # setattr:  setting "ccc" to value "0"
                    # getattribute:  attempting to access "__dict__"
                    # 0

setattr:  setting "a" to value "5"
getattribute:  attempting to access "__dict__"
getattribute:  attempting to access "a"
5
getattribute:  attempting to access "ccc"
getattr: "ccc" not found; setting default
setattr:  setting "ccc" to value "0"
getattribute:  attempting to access "__dict__"
0


In [47]:
class This(object):       # a simple class with one class variable
    a = 5

x = This()

print getattr(x, 'a')     # 5:  finds the 'a' attribute in the class

setattr(x, 'b', 10)       #     set x.b = 10 in the instance

print x.b                 # 10: retrieve x.b from the instance

5
10


#### @property: attribute control
This decorator allows behavior control when an individual attribute is accessed, through separate @property, @setter and @deleter methods.  

In [ ]:
class GetSet(object):

    def __init__(self,value):
        self.attrval = value

    @property # (read)
    def var(self):
        print 'getting the "var" attribute'
        return self.attrval

    @var.setter # sets property (write)
    def var(self, value):
        print 'setting the "var" attribute'
        self.attrval = value

    @var.deleter # deletes property (delete)
    def var(self):
        print 'deleting the "var" attribute'
        self.attrval = None

me = GetSet(5)
me.var
print me.var


me.var = 1000    # setting the "var" attribute
print me.var     # getting the "var" attribute
                 # 1000

del me.var       # deleting the "var" attribute
print me.var     # getting the "var" attribute
                 # None

An underscore (_) at the beginning is used to denote private variables in Python

In [79]:
#descriptors get and set values

class RevealAccess(object):
    """ A data descriptor that sets and returns values and prints a message declaring access. """

    def __init__(self, initval=None):
        self.val = initval

    def __get__(self, obj, objtype):
        print 'Getting attribute from object', obj
        print '...and doing some related operation that should take place at this time'
        return self.val

    def __set__(self, obj, val): # self (descriptor object = RevealAccess) # obj (MyClass = mm)
        print 'Setting attribute from object', obj
        print '...and doing some related operation that should take place at this time'
        self.val = val #obj.val instead?


# the class we will work with directly
class MyClass(object):
    """ A simple class with a class variable as descriptor """
    def __init__(self):
        print 'initializing object ', self

    x = RevealAccess(initval=0)  # attach a descriptor to class attribute 'x'
    # descriptors always have to be within a class
    # it is a class attribute because it is defined within a class (MyClass)


mm = MyClass()                   # initializing object  <__main__.MyClass object at 0x10066f7d0>

#mm.x = 5                         # Setting attribute from object <__main__.MyClass object at 0x1004de910>
                                 # ...and doing some related operation that should take place at this time

val = mm.x                       # Getting attribute from object <__main__.MyClass object at 0x1004de910>
                                 # ...and doing some related operation that should take place at this time

print 'retrieved value: ', val   # retrieved value:  5, and 0 if mm.x was never called so that default is 0

initializing object  <__main__.MyClass object at 0x1044b48d0>
Getting attribute from object <__main__.MyClass object at 0x1044b48d0>
...and doing some related operation that should take place at this time
retrieved value:  0


## Functional Programming

In [81]:

## The different paradigms


#procedural
# involves a series of statements along with variables that change as a result. 
# We call these variable values the program's state. 
mysum = 0
for counter in range(11):
    mysum = mysum + counter

print mysum

# OOP uses object state to produce outcome
class Summer(object):
    def __init__(self):
        self.sum = 0
    def add(self, num):
        self.sum = self.sum + num

s = Summer()
for num in range(11):
    s.add(num)

print s.sum

# functional programming combines both 
print sum(range(11))

55
55
55


In [ ]:
# LAMBDA examples

# sort a list of names by last name
names = [ 'Josh Peschko', 'Gabriel Feghali', 'Billy Woods', 'Arthur Fischer-Zernin' ]
sortednames = sorted(names, key=lambda name:  name.split()[1])

# sort a list of CSV lines by the 2nd column in the file
slines = sorted(lines, lambda x: x.split(',')[2])

### list comprehension

In [89]:
nums = [1, 2, 3, 4, 5]
dblnums = [ val * 2 for val in nums ]
print dblnums                                 # [2, 4, 6, 8, 10]

print [ val * 2 for val in nums if val > 2]   # [6, 8, 10] 

[2, 4, 6, 8, 10]
[6, 8, 10]


### set comprehension example

In [ ]:
states = { line.split(':')[3] 
           for line in open('student_db.txt').readlines()[1:] } # readlines()[1:] returns a set

### dict comprehension example

In [ ]:
student_states = { line.split(':')[0]: line.split(':')[3] 
                   for line in open('student_db.txt').readlines()[1:] }

### as an alternative to list comprehension, we can use map() and reduce()
- map(func, range) takes the function and applies it to the given sequence
- filter(criteria, range) applies criteria to given sequence and returns those that satisfy it
- reduce() applies a function to items in a sequence, but accumulates a value as it iterates.  

In [92]:
# square some integers
sqrd = map(lambda x: x ** 2, range(1,6))    # [1, 4, 9, 16, 25]
print sqrd
# get string lengths
lens = map(len, ['some', 'words', 'to', 'get', 'lengths', 'from'])
print lens    # [4, 5, 2, 3, 7, 4]

[1, 4, 9, 16, 25]
[4, 5, 2, 3, 7, 4]


In [93]:

pos = filter(lambda x: x > 0, [-5, 2, -3, 17, 6, 4, -9])
print pos     # [2, 17, 6, 4]

[2, 17, 6, 4]


In [127]:
def addthem(a, x):
    return a + x
intsum = reduce(addthem, range(1, 11))
print intsum

# same using a lambda
intsum = reduce(lambda a, x: a + x, range(1,11))
print intsum

55
55


In [130]:
# GENERATORS
# particularly useful in producing a sequence of n values
# i.e. not a fixed sequence, but an unlimited sequence.
# In this example we have prepared a generator that generates primes up to the specified limit.  

def get_primes(num_max):
    """ prime number generator """
    candidate = 2
    found = []
    while True:
        if all(candidate % prime != 0 for prime in found):
            yield candidate # yield -> returns values in a sequence one by one
            found.append(candidate)
        candidate += 1
        if candidate >= num_max:
            raise StopIteration

my_iter = get_primes(100)
print my_iter.next()        # 2
print my_iter.next()        # 3
print my_iter.next()        # 5

for i in get_primes(100):
    print i,

2
3
5
2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97


In [184]:
son = '''From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own bright eyes,
Feed'st thy light's flame with self-substantial fuel,
Making a famine where abundance lies,
Thy self thy foe, to thy sweet self too cruel:
Thou that art now the world's fresh ornament,
And only herald to the gaudy spring,
Within thine own bud buriest thy content,
And tender churl mak'st waste in niggarding:
Pity the world, or else this glutton be,
To eat the world's due, by the grave and thee.'''

sonnet = son.split(',')

def words(passage):
    for i in passage:
        yield i

for i in words(sonnet):
    print i

From fairest creatures we desire increase

That thereby beauty's rose might never die

But as the riper should by time decease

His tender heir might bear his memory:
But thou contracted to thine own bright eyes

Feed'st thy light's flame with self-substantial fuel

Making a famine where abundance lies

Thy self thy foe
 to thy sweet self too cruel:
Thou that art now the world's fresh ornament

And only herald to the gaudy spring

Within thine own bud buriest thy content

And tender churl mak'st waste in niggarding:
Pity the world
 or else this glutton be

To eat the world's due
 by the grave and thee.


Use recursion when there's an operation we need to repeat, but we don't know how many times

In [169]:
parents = {
            'Megacorp':           None,

            'Acme Office Supply': 'Megacorp',
            'Acme Paper':         'Acme Office Supply',
            'Acme Pens':          'Acme Office Supply',

            'Best Electronics':   None,
            'Best Audio':         'Best Electronics',
            'Best TV':            'Best Electronics',

            'Celera Publishing':  'Megacorp',
            'Celera Books':       'Celera Publishing',
            'Celera Web':         'Celera Publishing'
                                                        }

def top_parent(company_id):
    if ( company_id not in parents or       # base case (no parent):  return
                    not parents[company_id] ):

        return company_id

    return top_parent(parents[company_id])  # recursive call with parent id
    # calling top_parent from within top_parent -> publishing
    # if None, gives last returned value

for id in sorted(parents.keys()):
    print '{}:  {}'.format(id, top_parent(id))

Acme Office Supply:  Megacorp
Acme Paper:  Megacorp
Acme Pens:  Megacorp
Best Audio:  Best Electronics
Best Electronics:  Best Electronics
Best TV:  Best Electronics
Celera Books:  Megacorp
Celera Publishing:  Megacorp
Celera Web:  Megacorp
Megacorp:  Megacorp
